<a href="https://colab.research.google.com/github/KashishV999/nlp-transformers-journey/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning / Transfer learning
- by adding a new classification layer on top while keeping the pre-trained weights frozen
- Retains its language understanding gained from training , while learning task-specific patterns from the labeled dataset.

## LOAD DATASET

In [ ]:
!pip install -U datasets

In [ ]:
from datasets import load_dataset

In [ ]:
raw_datasets = load_dataset("SetFit/sst2")

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 6920
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 872
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1821
    })
})

In [ ]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'text': 'a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films',
 'label': 1,
 'label_text': 'positive'}

## TOKENIZE THE DATASET TO FEED TO MODEL

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"],truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6920
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## To compute the metrics

In [ ]:
!pip install evaluate

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

## comapre prediction with correct ans (label)
## - logits (raw score predicted)

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## TRAINING

In [ ]:
from transformers import TrainingArguments

In [ ]:

training_args = TrainingArguments(
    output_dir="test-trainer-sentiment",
     eval_strategy="epoch",
    report_to=[],

)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
train_output = trainer.train()

In [ ]:
train_output

TrainOutput(global_step=2595, training_loss=0.22319944929295651, metrics={'train_runtime': 402.9277, 'train_samples_per_second': 51.523, 'train_steps_per_second': 6.44, 'total_flos': 458828680555680.0, 'train_loss': 0.22319944929295651, 'epoch': 3.0})

## TESTING

In [ ]:
tokenized_datasets["test"]


Dataset({
    features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1821
})

In [ ]:
output = trainer.predict(tokenized_datasets["test"])


In [ ]:

output.metrics

{'test_loss': 0.4576760530471802,
 'test_accuracy': 0.9132344865458539,
 'test_runtime': 5.8218,
 'test_samples_per_second': 312.789,
 'test_steps_per_second': 39.163}